In [ ]:
import csv
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Reading in Dataset from JSON

The data we are using for our project is UCSD's Book Graph dataset. The main dataset is stored in a JSON with 2.3M entries and 29 fields, comprised of a variety of datatypes from integers and strings to dictionaries and lists. This JSON is too large to load into memory on colab, so we will process through the JSON line by line, appending the relevant data to a csv 10,000 lines at a time.

In [ ]:
import json
import gzip
import csv

fields_to_take = ['isbn', 'average_rating', 'description', 'link', 'authors',
                  'publisher', 'num_pages', 'isbn13', 'publication_year', 'image_url', 
                  'book_id', 'title', 'title_without_series', 'language_code'] #these are the fields that we want to keep
dict_chuck = []

i = 0
first = True
for line in gzip.open("gdrive/MyDrive/CIS 519 Project/Data/Raw Data/goodreads_books.json.gz", 'r'):
  json_line = json.loads(line.decode("utf-8"))
  line_dict = {}
  keep = False
  for field in fields_to_take: #for each field we want, extract the data associated with it and store in dict
    if field == "authors":
      if json_line[field] == []:
        out = ''
      else:
        out = json_line[field][0]['author_id']
    #in the image_url datafield, Good Reads uses a default image when it doesn't have access to a cover; we want to remove these
    elif field == "image_url" and json_line[field] == 'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png':
      out = '' 
    elif field == "language_code": #if the language is set to a langauage other than english, we aren't using it in our analysis 
      if json_line[field] == "eng" or json_line[field] == "":
        keep = True
        out = json_line[field]
    else:
      out = json_line[field]

    line_dict[field] = out
  if keep: 
    dict_chuck.append(line_dict)

  i+=1

  # append rows every 10000 lines
  if i % 10000 == 0:
    print(i)
    if first:
      with open('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fields_to_take)
        writer.writeheader()
        writer.writerows(dict_chuck)
      first = False
      dict_chuck = []
    else:
      with open('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv', 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = line_dict.keys())
        writer.writerows(dict_chuck)
      dict_chuck = []


After we extract all of the data we want, we're left with the following csv with 1,768,097 rows and 14 columns:

In [ ]:
import pandas as pd
df = pd.read_csv('gdrive/MyDrive/CIS 519 Project/Data/goodreads_books_cleaned.csv')
df.head()

,isbn,average_rating,description,link,authors,publisher,num_pages,isbn13,publication_year,image_url,book_id,title,title_without_series,language_code
0,0312853122,4.00,NaN,https://www.goodreads.com/book/show/5333265-w-...,604031.0,St. Martin's Press,256.0,9780312853129,1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,NaN
1,0743509986,3.23,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,626222.0,Simon & Schuster Audio,NaN,9780743509985,2001.0,NaN,1333909,Good Harbor,Good Harbor,NaN
2,NaN,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,10333.0,"Nelson Doubleday, Inc.",600.0,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",eng
3,0743294297,3.49,Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,9212.0,Atria Books,368.0,9780743294294,2009.0,NaN,6066819,Best Friends Forever,Best Friends Forever,eng
4,0850308712,3.40,NaN,https://www.goodreads.com/book/show/287140.Run...,149918.0,NaN,NaN,9780850308716,NaN,https://images.gr-assets.com/books/1413219371m...,287140,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,NaN


# Genre Identification Dataset

The next task we have is to include the genres in our data. The UCSD data pulls genre from user tags in the following format:

In [ ]:
import pandas as pd
genre_created = pd.read_json('gdrive/MyDrive/CIS 519 Project/Data/Raw Data/goodreads_book_genres_initial.json.gz',lines=True)
genre_created.head()

,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


We then clean this data and process it into a usable format by sorting the genres by the number of tags and convert them into a columns where the 'first' column has the genre most often tagged, the 'second' column has the genre tagged the second most, etc. We also want to explore different genre conventions, so we add in the 'genre_cleaned' column which uses the second most common tag in place of the first if the first tag is 'fiction'. This is done in an attempt to make the genres more specific. In future parts of the project, models are trained against the true most common genre ("first") as well as this created columns.

In [ ]:
def cleanGenreList(originalList):
  out = sorted(dict(originalList['genres']), key=originalList['genres'].get, reverse=True)
  if len(out) == 0:
    out =  ['','','']
  elif len(out) == 1:
    out.append('')
    out.append('')
  elif len(out) == 2:
    out.append('')
  else:
    out = out[:3]
  return out


In [ ]:
genre_created_cleaned = genre_created.copy()
genre_created_cleaned[['first', 'second', 'third']] = genre_created.apply(lambda x: cleanGenreList(x), axis=1, result_type='expand')
genre_created_cleaned.head()

,book_id,genres,first,second,third
0,5333265,"{'history, historical fiction, biography': 1}","history, historical fiction, biography",,
1,1333909,"{'fiction': 219, 'history, historical fiction,...",fiction,"history, historical fiction, biography",
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys...","fantasy, paranormal",fiction,"mystery, thriller, crime"
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri...",fiction,romance,"mystery, thriller, crime"
4,287140,{'non-fiction': 3},non-fiction,,


In [ ]:
def cleanGenre(row):
  if row['first'] == 'fiction' and row['second'] != '':
    return row['second']
  else:
    return row['first']

In [ ]:
genre_created_cleaned["genre_cleaned"] = genre_created_cleaned.apply(lambda x: cleanGenre(x), axis=1)
genre_created_cleaned.head()

,book_id,genres,first,second,third,genre_cleaned
0,5333265,"{'history, historical fiction, biography': 1}","history, historical fiction, biography",,,"history, historical fiction, biography"
1,1333909,"{'fiction': 219, 'history, historical fiction,...",fiction,"history, historical fiction, biography",,"history, historical fiction, biography"
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys...","fantasy, paranormal",fiction,"mystery, thriller, crime","fantasy, paranormal"
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri...",fiction,romance,"mystery, thriller, crime",romance
4,287140,{'non-fiction': 3},non-fiction,,,non-fiction


In [14]:
genre_lookup[['first','book_id']].groupby(by=['first']).count()

,book_id
first,
children,116941
"comics, graphic",91185
"fantasy, paranormal",221908
fiction,435045
"history, historical fiction, biography",177837
"mystery, thriller, crime",194964
non-fiction,335632
poetry,43175
romance,276087


In [12]:
genre_lookup[['genre_cleaned','book_id']].groupby(by=['genre_cleaned']).count()

,book_id
genre_cleaned,
children,129873
"comics, graphic",93720
"fantasy, paranormal",299786
fiction,83201
"history, historical fiction, biography",296874
"mystery, thriller, crime",243219
non-fiction,346963
poetry,50070
romance,329560


In [5]:
genre_created_cleaned.to_csv('gdrive/MyDrive/CIS 519 Project/Data/genre_lookup_cleaned.csv')

#from google.colab import drive
#drive.mount('/content/gdrive')
#import pandas as pd
#genre_lookup = pd.read_csv('gdrive/MyDrive/Grad School/Spring 2022/CIS 519/CIS 519 Project/Data/Raw Data/OLD_genre_lookup_cleaned.csv')

In the genre_cleaned column, we can see that the number of books tagged as 'fiction' has decreased from 435K to 83K.



---



# Joined together

Next we need to join these files together

In [ ]:
df_w_genre = df.merge(genre_lookup, how="inner", on="book_id")
df_w_genre = df_w_genre.drop(columns=['Unnamed: 0', 'genres'])
df_w_genre.head()

,isbn,average_rating,description,link,authors,publisher,num_pages,isbn13,publication_year,image_url,book_id,title,title_without_series,language_code,first,second,third,genre_cleaned
0,0312853122,4.00,NaN,https://www.goodreads.com/book/show/5333265-w-...,604031.0,St. Martin's Press,256.0,9780312853129,1984.0,https://images.gr-assets.com/books/1310220028m...,5333265,W.C. Fields: A Life on Film,W.C. Fields: A Life on Film,NaN,"history, historical fiction, biography",NaN,NaN,"history, historical fiction, biography"
1,0743509986,3.23,"Anita Diamant's international bestseller ""The ...",https://www.goodreads.com/book/show/1333909.Go...,626222.0,Simon & Schuster Audio,NaN,9780743509985,2001.0,NaN,1333909,Good Harbor,Good Harbor,NaN,fiction,"history, historical fiction, biography",NaN,"history, historical fiction, biography"
2,NaN,4.03,Omnibus book club edition containing the Ladie...,https://www.goodreads.com/book/show/7327624-th...,10333.0,"Nelson Doubleday, Inc.",600.0,NaN,1987.0,https://images.gr-assets.com/books/1304100136m...,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","The Unschooled Wizard (Sun Wolf and Starhawk, ...",eng,"fantasy, paranormal",fiction,"mystery, thriller, crime","fantasy, paranormal"
3,0743294297,3.49,Addie Downs and Valerie Adler were eight when ...,https://www.goodreads.com/book/show/6066819-be...,9212.0,Atria Books,368.0,9780743294294,2009.0,NaN,6066819,Best Friends Forever,Best Friends Forever,eng,fiction,romance,"mystery, thriller, crime",romance
4,0850308712,3.40,NaN,https://www.goodreads.com/book/show/287140.Run...,149918.0,NaN,NaN,9780850308716,NaN,https://images.gr-assets.com/books/1413219371m...,287140,Runic Astrology: Starcraft and Timekeeping in ...,Runic Astrology: Starcraft and Timekeeping in ...,NaN,non-fiction,NaN,NaN,non-fiction


In [ ]:
df_w_genre.to_csv('gdrive/MyDrive/CIS 519 Project/Data/books_with_genre.csv.gz', compression='gzip')

# Cleaning

Closer inspection of the data shows that some rows missing the language tags are in a language other than English. As the scope of our project only extends to English language books, we need to identify and remove them.

## Cleaning Language

In [ ]:
#!pip install langdetect
from langdetect import detect

In [ ]:
def detectLang(x):
  if x != x:
    return ''
  elif type(x) == str:
    try:
      return detect(x)
    except:
      print('') #if the description cannot be read by the language detection package, skip it. Most of these are only punctuation or links
  else:
    print(type(x))


In [ ]:
df_w_genre['detected_lang'] = df_w_genre.loc[(df_w_genre['language_code'] != df_w_genre['language_code']) & \
                                             (df_w_genre['description'] == df_w_genre['description']) & \
                                             (~df_w_genre['description'].isin(['<>', '<', '>', '.', ','])), 'description']\
                                             .apply(detectLang)



Now that all the rows in question have a language assigned, what do they look like?


In [ ]:
from collections import Counter

Counter(df_w_genre['detected_lang'])

Counter({None: 145,
         'af': 129,
         'ca': 135,
         'cs': 170,
         'cy': 4343,
         'da': 347,
         'de': 2877,
         'en': 754055,
         'es': 7630,
         'et': 284,
         'fi': 424,
         'fr': 4041,
         'hr': 361,
         'hu': 85,
         'id': 1563,
         'it': 1652,
         'lt': 98,
         'lv': 40,
         nan: 984276,
         'nl': 809,
         'no': 189,
         'pl': 353,
         'pt': 1454,
         'ro': 277,
         'sk': 184,
         'sl': 418,
         'so': 303,
         'sq': 108,
         'sv': 431,
         'sw': 66,
         'tl': 156,
         'tr': 537,
         'vi': 157})

Most of the nans in this list were values that were assigned English by the dataset and thus skipped in the last step. The values of None are the values that were exceptions in the language assignment code so we want to remove those.

In [ ]:
df_w_genre = df_w_genre[~df_w_genre['detected_lang'].isin([None])]
df_w_genre = df_w_genre[df_w_genre['first'] == df['first']]

## Assigning other Genre Options
As discussed above, we want to include several options to use as labels to find the best model that we can. Here, we are creating two more genre columns, one of which concatenates the second genre with the first if the first is fiction. The other, genre_cleaned3, appends the word 'fiction' to the front of the first genre if the second genre is fiction. Examples are included below. 

We also only want to include books that were either initially identified as english by the UCSD dataset, were identified as english by the language identification package above or don't have descriptions at all. This might leave some non-english book titles but those will likely not effect the models too deeply.

In [ ]:
def cleanGenre2(row):
  if row['first'] == 'fiction' and row['second'] == row['second']:
    return (row['first']+": "+row['second'])
  else:
    return row['first']


df_w_genre['genre_cleaned2'] = df_w_genre.apply(cleanGenre2, axis=1)

In [ ]:
def cleanGenre3(row):
  if row['second'] == 'fiction':
    return (row['second']+": "+row['first'])
  else:
    return row['genre_cleaned2']


df_w_genre['genre_cleaned3'] = df_w_genre.apply(cleanGenre3, axis=1)

In [ ]:
df_w_genre = df_w_genre[(df_w_genre['language_code'] == 'eng') | (df_w_genre['detected_lang'] == 'en') |\
                        ((df_w_genre['description'] != df_w_genre['description']) & \
                         (df_w_genre['detected_lang'] != df_w_genre['detected_lang'] ))]
df_w_genre = df_w_genre.drop(columns=['language_code', 'detected_lang', 'Unnamed: 0'])
df_w_genre.to_csv('gdrive/MyDrive/CIS 519 Project/Data/books_clean.csv.gz', compression='gzip')


In [20]:
#example of genre_cleaned

df_w_genre[df_w_genre['first'] != df_w_genre['genre_cleaned']].head(1)[['first','second','third','genre_cleaned']]

,first,second,third,genre_cleaned
1,fiction,"history, historical fiction, biography",NaN,"history, historical fiction, biography"


In [21]:
#example of genre_cleaned2

df_w_genre[df_w_genre['first'] != df_w_genre['genre_cleaned']].head(1)[['first','second','third','genre_cleaned2']]

,first,second,third,genre_cleaned2
1,fiction,"history, historical fiction, biography",NaN,"fiction: history, historical fiction, biography"


In [24]:
#example of genre_cleaned3

df_w_genre[df_w_genre['genre_cleaned2'] != df_w_genre['genre_cleaned3']].head(1)[['first','second','third','genre_cleaned3']]

,first,second,third,genre_cleaned3
2,"fantasy, paranormal",fiction,"mystery, thriller, crime","fiction: fantasy, paranormal"
